<hr style="border:1px solid gray"> </hr>

# Lecture: What is Homotopy Continuation?

Homotopy Continuation is a computational framework for solving systems of polynomials equations over the complex numbers $\mathbb C$.

## Example:
Polynomials in variables $x,y$: 

\begin{align*}
\text{❌} \quad g(x,y) &= 13 \exp(x) + y + 1\\[0.5em]
\text{❌} \quad h(x,y) &= \sqrt{x+y} - y^2\\[0.5em]
\text{✅} \quad f(x,y) &= 4x^2 + 2xy - (10 + 4i) (y-1)\\
\end{align*}



In the following, let

$$F(x_1,\ldots,x_n) = \begin{bmatrix} f_1(x_1,\ldots,x_n)\\ \vdots \\ f_m(x_1,\ldots,x_m)\end{bmatrix}$$

be a **system** of $m$ polynomials in $n$ variables.

* If $n=m$, we call $F$ a <u>square system</u>
* If $n<m$, we call $F$ <u>overdetermined</u>
* If $n>m$, we call $F$ <u>underdetermined</u>

<hr style="border:1px solid gray"> </hr>

# What solving means

In the context of homotopy continuation *solving* means to compute **numerical approximations** of **isolated solutions**.

Let us look at an example: take 

$$F(x,y,z) = \begin{bmatrix} x^2 + y^2 + z^2 - 1\\ x^2 - y + z^2\\ x-z\end{bmatrix}.$$

A useful representation of $F=0$ is by expressing it with a *Gröbner basis*:

$$ \mathcal G = \{x-z, \; y-2z^2, \; z^4+\tfrac{1}{2} z^2 - \tfrac{1}{4}\}.$$

We can read off from $\mathcal G$ the number of solutions of $F=0$. 

And we can solve $F=0$ by iteratively solving *univariate equation*. 

<br>

But this is a reduction to the problem we started with: solving equations.

Homotopy Continuation takes a different approach. 

It keeps the multivariate structure and tries directly to compute points $\xi\in\mathbb C^n$ such that the distance $\Vert \xi-\zeta\Vert$ to a true zero $\zeta$ with $F(\zeta)=0$ is small.

In [32]:
using HomotopyContinuation

@var x y z
F = [x^2 + y^2 + z^2 - 1; x^2 - y + z^2; x-z]

zeros = solve(F)

Result with 4 solutions
• 4 paths tracked
• 4 non-singular solutions (2 real)
• random_seed: 0x8b97d294
• start_system: :polyhedral


In [25]:
[round.(s, digits = 3) for s in solutions(zeros)]

4-element Array{Array{Complex{Float64},1},1}:
 [-0.0 - 0.899im, -1.618 - 0.0im, 0.0 - 0.899im]
 [0.556 + 0.0im, 0.618 - 0.0im, 0.556 + 0.0im]
 [-0.556 + 0.0im, 0.618 + 0.0im, -0.556 - 0.0im]
 [0.0 + 0.899im, -1.618 - 0.0im, -0.0 + 0.899im]

The true solutions are

\begin{align*}
(x,y,z) &= (\tfrac{1}{2}\sqrt{\sqrt{5} -1},\; \tfrac{1}{2}(\sqrt{5} -1),\; \tfrac{1}{2}\sqrt{\sqrt{5} -1}\,)\\[0.5em]
(x,y,z) &= (-\tfrac{1}{2}\sqrt{\sqrt{5} -1},\; \tfrac{1}{2}(\sqrt{5} -1),\; -\tfrac{1}{2}\sqrt{\sqrt{5} -1}\,)\\[0.5em]
(x,y,z) &= (\tfrac{i}{2}\sqrt{\sqrt{5} +1},\; -(\sqrt{5} +1),\;\tfrac{i}{2}\sqrt{\sqrt{5} +1}\,)\\[0.5em]
(x,y,z) &= (-\tfrac{i}{2}\sqrt{\sqrt{5} +1},\; -(\sqrt{5} +1),\;-\tfrac{i}{2}\sqrt{\sqrt{5} +1}\,)\\
\end{align*}

<br>

Why do we even compute numerical approximations when we can have such exact results?

<u>Answer:</u> numerical methods are often much faster, especially when the problems become more complicated.

<hr style="border:1px solid gray"> </hr>


# The basic idea

We first discuss the basic idea underlying homotopy continuation. For this, we denote 

$$\mathbb C[x_1,\ldots,x_n]_{d} := \{\text{polynomials with coefficients in $\mathbb C$ of degree at most $d$ in the variables $x_1,\ldots,x_n$}\}.$$

Let $d_1,\ldots,d_m$ be fixed and define

$$\mathcal R := \mathbb C[x_1,\ldots,x_n]_{d_1} \times \cdots \times \mathbb C[x_1,\ldots,x_n]_{d_m}$$

to be the vector space of systems of $m$ polynomials in $n$ variables with complex coefficients.

Suppose that the <u>system we are interested in</u> is 

$$F(x) \in \mathcal R.$$

Suppose further that there is another system

$$G(x) \in \mathcal R,$$

of which we know or can easily compute a zero $\zeta$ with $G(\zeta)=0$. Let $H(x,t): \mathcal R \times [0,1] \to \mathcal R$
be a **homotopy** in $\mathcal R$ with 

$$H(x,1) = G(x)\quad \text{and}\quad H(x,0)=F(x).$$ 



<bdi style="background-color:powderblue;">
The idea is to track $\zeta$ along the homotopy $H(x,t)$ from $t=1$ to $t=0$.
</bdi>

<img src="tracking.gif" width="500" style="float:right;">

<img src="geometry.png" width="400"> 

## Example

$n=m=1$ and $G(x)=x^8 -1$.

The zeros of $G$ are $\{\exp(2\pi i/8)\mid 1\leq i\leq 8\}$.

(points on the unit circle)

As $t$ moves from $1$ to $0$ the blue points move in the complex plane $\mathbb C$.

<hr style="border:1px solid gray"> </hr>

# What tracking means

Tracking means to compute a numerical approximation of the **solution curve** $x(t)$ with

$$H(x(t), t) = 0.$$

Differentiating at $t$ gives:

$$\frac{\partial}{\partial t} H(x,t) + \frac{\partial}{\partial x} H(x,t) \,\frac{\partial}{\partial t} x(t) = 0.$$


Computing $x(t)$ is equivalent to solving an 
<bdi style="background-color:powderblue;">
ODE initial value problem!
</bdi>

<br>

Numerical approximation means that we want to compute discrete values $t_1,t_2,\ldots,t_k,\ldots,$ and points $\widetilde{x}_1,\widetilde{x}_2,\ldots,\widetilde{x}_k,\ldots$ such that $\Vert \widetilde{x}_k - x(t_k)\Vert$ is small for all $k$. Then, we can move $\widetilde{x}_k$ towards $x(t_k)$ using *Newton's method*. This is called the <bdi style="background-color:powderblue;">corrector</bdi>.

<img src="predictor-corrector.png" width="500"> 

The <bdi style="background-color:powderblue;">predictor</bdi> produces $\widetilde{x}_{k}$ from an approximation of $x(t_{k-1})$. 


There are many predictor methods for ODEs. HomotopyContinuation.jl uses a variant of the so called Padé-predictor

(Sascha and Simon can tell more about this).

<hr style="border:1px solid gray"> </hr>

# Gotta catch them all

So far, the discussion evolved around tracking a single solution curve and hence computing a single solutions.

Polynomial homotopy continuation can do more: it can compute **all solutions**.

Recall the definition 

$$\mathbb C[x_1,\ldots,x_n]_{d} := \{\text{polynomials with coefficients in $\mathbb C$ of degree at most $d$ in the variables $x_1,\ldots,x_n$}\}.$$

and

$$\mathcal R := \mathbb C[x_1,\ldots,x_n]_{d_1} \times \cdots \times \mathbb C[x_1,\ldots,x_n]_{d_m}.$$

Suppose $F(x)\in\mathcal R$. Then, we can take the following as start system

$$G(x) = \begin{bmatrix} x_1^{d_1} - 1\\ \vdots \\ x_n^{d_n} - 1\end{bmatrix}.$$

$G(x)$ is called a **total degree start system**.

<u> **Theorem**</u> (Bézout's theorem): $F(x)=0$ has at most $d_1\cdots d_n$ isolated solutions.

<br>

This means that $G(x)$ has at least as many isolated solutions as $F(x)$. 

If we can find for each zero $F(\zeta)=0$ a corresponding zero $G(\xi)=0$, such that $\xi$ gets tracked towards $\zeta,$ we can compute all solutions of $F(x)=0$.

<br>


**Problem 1:** 
<bdi style="background-color:powderblue;">
Many zeros of $G(x)$ do not get tracked towards a zero of $F$. Computation is wasteful!
</bdi>

In [35]:
@var x y
F = [x^2 - x^2 * y^2 + 2y^2 - 1;  x - 4x * y + 3]
solve(F, start_system = :total_degree)

Result with 4 solutions
• 8 paths tracked
• 4 non-singular solutions (0 real)
• random_seed: 0xaecaa79f
• start_system: :total_degree


<br>

**Problem 2:** 
<bdi style="background-color:powderblue;">
Which homotopy to take?
</bdi>

It is appealing to take the *straight-line homotopy* $H(x,t) = tF(x) + (1-t)G(x)$.

It is, however, better to take a **path in the complex numbers**, because predictor-corrector methods rely on  $\frac{\partial}{\partial x} H(x,t)$ being invertible. 

The space of polynomial systems in $\mathcal R$, whose derivative is *not* invertible, form an **algebraic subvariety** of $\mathcal R$. 

* In $\mathcal R\cap \{\text{systems with real coefficients}\}$ it has real codimension $1$.

* In $\mathcal R$ it has real codimension $2$. We can go around it!

<hr style="border:1px solid gray"> </hr>

# Start systems

The start system $G(x) = [x_i^{d_i} - 1]$ is not always an optimal choice.

<br>

<bdi style="background-color:powderblue;">
Are there better choices?</bdi>

Yes, but the difficulty is constructing them.




